<a href="https://colab.research.google.com/github/MDfarazuddin99/CNN_Pruning/blob/master/LeNet5_MNIST_EUCLIDEAN_DISTANCE_PRUNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%tensorflow_version 1.x
# !pip install tesnsorflow 1.x
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import keras
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, GlobalAveragePooling2D,BatchNormalization,Activation,AveragePooling2D
from keras.models import load_model
from keras.callbacks import Callback
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.layers.core import Lambda
from keras import backend as K
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from sklearn import preprocessing


!pip install kerassurgeon
from kerassurgeon import identify 
from kerassurgeon.operations import delete_channels,delete_layer
from kerassurgeon import Surgeon

In [0]:
def my_get_all_conv_layers(model , first_time):

    '''
    Arguments:
        model -> your model
        first_time -> type boolean 
            first_time = True => model is not pruned 
            first_time = False => model is pruned
    Return:
        List of Indices containing convolution layers
    '''

    all_conv_layers = list()
    for i,each_layer in enumerate(model.layers):
        if (each_layer.name[0:6] == 'conv2d'):
            all_conv_layers.append(i)
    return all_conv_layers if (first_time==True) else all_conv_layers[1:]


def my_get_all_dense_layers(model):
    '''
    Arguments:
        model -> your model        
    Return:
        List of Indices containing fully connected layers
    '''
    all_dense_layers = list()
    for i,each_layer in enumerate(model.layers):
        if (each_layer.name[0:5] == 'dense'):
            all_dense_layers.append(i)
    return all_dense_layers



def my_get_weights_in_conv_layers(model,first_time):

    '''
    Arguments:
        model -> your model
        first_time -> boolean variable
            first_time = True => model is not pruned 
            first_time = False => model is pruned
    Return:
        List containing weight tensors of each layer
    '''
    weights = list()
    all_conv_layers = my_get_all_conv_layers(model,first_time)
    layer_wise_weights = list() 
    for i in all_conv_layers:
          weights.append(model.layers[i].get_weights()[0])  
    return weights

def my_get_l1_norms_filters_per_epoch(weight_list_per_epoch):

    '''
    Arguments:
        List
    Return:
        Number of parmaters, Number of Flops
    '''
    
    # weight_list_per_epoch = my_get_weights_in_conv_layers(model,first_time)
    l1_norms_filters_per_epoch = list()
    

    for index in range(len(weight_list_per_epoch)):

        epochs = np.array(weight_list_per_epoch[index]).shape[0]
        h , w , d = np.array(weight_list_per_epoch[index]).shape[1], np.array(weight_list_per_epoch[index]).shape[2] , np.array(weight_list_per_epoch[index]).shape[3]


        l1_norms_filters_per_epoch.append(np.sum(np.abs(np.array(weight_list_per_epoch[index])).reshape(epochs,h*w*d,-1),axis=1))
    return l1_norms_filters_per_epoch

def my_in_conv_layers_get_sum_of_l1_norms_sorted_indices(weight_list_per_epoch):
    '''
        Arguments:
            weight List 
        Return:
            
    '''
    layer_wise_filter_sorted_indices = list()
    layer_wise_filter_sorted_values = list()
    l1_norms_filters_per_epoch = my_get_l1_norms_filters_per_epoch(weight_list_per_epoch)
    sum_l1_norms = list()
    
    for i in l1_norms_filters_per_epoch:
        sum_l1_norms.append(np.sum(i,axis=0))
    
    layer_wise_filter_sorted_indices = list()
    
    for i in sum_l1_norms:
        a = pd.Series(i).sort_values().index
        layer_wise_filter_sorted_indices.append(a.tolist())
    return layer_wise_filter_sorted_indices


def my_get_percent_prune_filter_indices(layer_wise_filter_sorted_indices,percentage):    

    prune_filter_indices = list()
    for i in range(len(layer_wise_filter_sorted_indices)):
        prune_filter_indices.append(int(len(layer_wise_filter_sorted_indices[i]) * (percentage/100)))
    return prune_filter_indices

def my_get_distance_matrix(l1_norm_matrix):
    distance_matrix = []
    for i,v1 in enumerate(l1_norm_matrix):
        distance_matrix.append([])
        for v2 in l1_norm_matrix:
            distance_matrix[i].append(np.sum(np.abs((v1-v2))))
    return np.array(distance_matrix)
    
def my_get_distance_matrix_list(l1_norm_matrix_list):
    distance_matrix_list = []
    for l1_norm_matrix in l1_norm_matrix_list:
        distance_matrix_list.append(my_get_distance_matrix(l1_norm_matrix.T))
    return distance_matrix_list



def my_get_episodes(distance_matrix,percentage):
    distance_matrix_flatten = pd.Series(distance_matrix.flatten())
    
    distance_matrix_flatten = distance_matrix_flatten.sort_values().index.to_list()
    
    episodes = list()
    n = distance_matrix.shape[0]
    for i in distance_matrix_flatten:
        episodes.append((i//n,i%n))
    k = int((n * percentage)/100)
    li = list()   
    for i in range(2*k):
        if i%2!=0:
            li.append(episodes[n+i])
    return li


def my_get_episodes_for_all_layers(distance_matrix_list,percentage):
    all_episodes = list()
    for matrix in distance_matrix_list:
        all_episodes.append(my_get_episodes(matrix,percentage))
    return all_episodes


def my_get_filter_pruning_indices(episodes_for_all_layers,l1_norm_matrix_list):
    filter_pruning_indices = list()
    for layer_index,episode_layer in enumerate(episodes_for_all_layers):
        a = set()
        sum_l1_norms = np.sum(l1_norm_matrix_list[layer_index],axis=0,keepdims=True)

        for episode in episode_layer:
            ep1 = sum_l1_norms.T[episode[0]]
            ep2 = sum_l1_norms.T[episode[1]]
            if ep1 >= ep2:
                a.add(episode[0])
            else:
                a.add(episode[1])
            a.add(episode[0])
        a = list(a)
        filter_pruning_indices.append(a)
    return filter_pruning_indices


    
def my_delete_filters(model,weight_list_per_epoch,percentage,first_time,keep_prob):
    l1_norms = my_get_l1_norms_filters_per_epoch(weight_list_per_epoch)
    distance_matrix_list = my_get_distance_matrix_list(l1_norms)
    episodes_for_all_layers = my_get_episodes_for_all_layers(distance_matrix_list,percentage)
    filter_pruning_indices = my_get_filter_pruning_indices(episodes_for_all_layers,l1_norms)
    all_conv_layers = my_get_all_conv_layers(model,first_time)

    surgeon = Surgeon(model)
    for index,value in enumerate(all_conv_layers):
        # print(index,value,filter_pruning_indices[index])
        if np.random.rand(1) < keep_prob :
            surgeon.add_job('delete_channels',model.layers[value],channels = filter_pruning_indices[index])
    model_new = surgeon.operate()
    return model_new    


def count_conv_params_flops(conv_layer):
    # out shape is  n_cells_dim1 * (n_cells_dim2 * n_cells_dim3)
    '''
    Arguments:
        conv layer 
    Return:
        Number of Parameters, Number of Flops
    '''
    
    
    out_shape = conv_layer.output_shape

    n_cells_total = np.prod(out_shape[1:-1])

    n_conv_params_total = conv_layer.count_params()
    # print(n_conv_params_total,len(conv_layer.get_weights()[0]),)
    conv_flops = 2 * (n_conv_params_total * n_cells_total - len(conv_layer.get_weights()[1]) *n_cells_total)

 

    return n_conv_params_total, conv_flops


def count_dense_params_flops(dense_layer):
    # out shape is  n_cells_dim1 * (n_cells_dim2 * n_cells_dim3)
    '''
    Arguments:
      dense layer 
    Return:
        Number of Parameters, Number of Flops
    '''

    out_shape = dense_layer.output_shape
    n_cells_total = np.prod(out_shape[1:-1])

    n_dense_params_total = dense_layer.count_params()

    dense_flops = 2* (n_dense_params_total - len(dense_layer.get_weights()[1]) * n_cells_total)


    return n_dense_params_total, dense_flops




def count_model_params_flops(model,first_time):

    '''
    Arguments:
        model -> your model
        first_time -> boolean variable
        first_time = True => model is not pruned 
        first_time = False => model is pruned
    Return:
        Number of parmaters, Number of Flops
    '''

    total_params = 0
    total_flops = 0
    # if first_time == True:
    #     model_layers = model.layers[:-3]
    # else:
    #     model_layers = model.layers[1:-3]
    model_layers = model.layers
    for index,layer in enumerate(model_layers):
        if any(conv_type in str(type(layer)) for conv_type in ['Conv1D', 'Conv2D', 'Conv3D']):
            
            params, flops = count_conv_params_flops(layer)
            # print(index,layer.name,params,flops)
            total_params += params
            total_flops += flops
        elif 'Dense' in str(type(layer)):
            
            params, flops = count_dense_params_flops(layer)
            # print(index,layer.name,params,flops)
            total_params += params
            total_flops += flops
    return total_params, int(total_flops)



In [0]:
class Get_Weights(Callback):
    def __init__(self,first_time):
        super(Get_Weights, self).__init__()
        self.weight_list = [] #Using a list of list to store weight tensors per epoch
        self.first_time = first_time
    def on_epoch_end(self,epoch,logs=None):
        if epoch == 0:
            all_conv_layers = my_get_all_conv_layers(self.model,self.first_time)
            for i in range(len(all_conv_layers)):
                self.weight_list.append([]) # appending empty lists for later appending weight tensors 
        
        for index,each_weight in enumerate(my_get_weights_in_conv_layers(self.model,self.first_time)):
                self.weight_list[index].append(each_weight)

In [6]:
model = keras.Sequential()

model.add(Conv2D(filters=20, kernel_size=(5, 5), activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D())

model.add(Conv2D(filters=50, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(units=500, activation='relu'))

model.add(Dense(units=10, activation = 'softmax'))

def train(model,epochs,first_time):
    (x_train, y_train), (x_test, y_test) = mnist.load_data()


    img_rows, img_cols = 28, 28
    batch_size = 128
    num_classes = 10

    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255




    y_train = tf.keras.utils.to_categorical(y_train, num_classes)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes)


    # Compile the model
    adam = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy']) 

    gw = Get_Weights(first_time)
    history = model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=2,
            callbacks=[gw],
            validation_data=(x_test, y_test))

    return model,history,gw.weight_list

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 20)        520       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 50)          25050     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               400500    
_________________________________________________________________
dense_2 (Dense)              (None, 10)               

In [8]:
count_model_params_flops(model,True)

(431080, 4586000)

In [9]:
model,history,weight_list_per_epoch = train(model,10,True)
initial_flops = count_model_params_flops(model,True)[1]
log_dict = dict()
log_dict['train_loss'] = []
log_dict['train_acc'] = []
log_dict['val_loss'] = []
log_dict['val_acc'] = []
log_dict['total_params'] = []
log_dict['total_flops'] = []
log_dict['filters_in_conv1'] = []
log_dict['filters_in_conv2'] = []

best_acc_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
log_dict['train_loss'].append(history.history['loss'][best_acc_index])
log_dict['train_acc'].append(history.history['accuracy'][best_acc_index])
log_dict['val_loss'].append(history.history['val_loss'][best_acc_index])
log_dict['val_acc'].append(history.history['val_accuracy'][best_acc_index])
a,b = count_model_params_flops(model,True)
log_dict['total_params'].append(a)
log_dict['total_flops'].append(b)
log_dict['filters_in_conv1'].append(model.layers[0].get_weights()[0].shape[-1])
log_dict['filters_in_conv2'].append(model.layers[2].get_weights()[0].shape[-1])

11493376/11490434 [==============================] - 1s 0us/step



Train on 60000 samples, validate on 10000 samples
Epoch 1/10
 - 9s - loss: 0.1810 - accuracy: 0.9442 - val_loss: 0.0481 - val_accuracy: 0.9847
Epoch 2/10
 - 2s - loss: 0.0488 - accuracy: 0.9846 - val_loss: 0.0389 - val_accuracy: 0.9874
Epoch 3/10
 - 2s - loss: 0.0346 - accuracy: 0.9890 - val_loss: 0.0325 - val_accuracy: 0.9893
Epoch 4/10
 - 2s - loss: 0.0258 - accuracy: 0.9918 - val_loss: 0.0272 - val_accuracy: 0.9903
Epoch 5/10
 - 2s - loss: 0.0204 - accuracy: 0.9932 - val_loss: 0.0289 - val_accuracy: 0.9905
Epoch 6/10
 - 2s - loss: 0.0144 - accuracy: 0.9954 - val_loss: 0.0250 - val_accuracy: 0.9920
Epoch 7/10
 - 2s - loss: 0.0119 - accuracy: 0.9960 - val_loss: 0.0335 - val_accuracy: 0.9901
Epoch 8/10
 - 2s - loss: 0.0103 - accuracy: 0.9967 - val_loss: 0.0258 - val_accuracy: 0.9927
Epoch 9/10
 - 2s - loss: 0.0094 - accuracy: 0.9967 - val_loss: 0.0390 - val_accuracy: 0.9888
Epoch 10/10
 - 2s - loss: 0.0075 - accuracy: 0.9974 - val_loss: 0.0393 - val_accuracy: 0.9899


In [10]:
print('Validation accuracy ',max(history.history['val_accuracy']))

Validation accuracy  0.9926999807357788


In [11]:

#stop pruning if the accuracy drops by 5% from maximum accuracy ever obtained. 
validation_accuracy = max(history.history['val_accuracy'])
print("Initial Validation Accuracy = {}".format(validation_accuracy) )
max_val_acc = validation_accuracy
count = 0
all_models = list()
a,b = count_model_params_flops(model,False)
print(a,b)
while validation_accuracy - max_val_acc >= -0.01 and  count < 3:


    print("ITERATION {} ".format(count+1))
    all_models.append(model)
    if max_val_acc < validation_accuracy:
        max_val_acc = validation_accuracy
        

    if count < 1:
        model = my_delete_filters(model,weight_list_per_epoch,50,True,1)
        model,history,weight_list_per_epoch = train(model,10,False)
   
    else:
        model = my_delete_filters(model,weight_list_per_epoch,20,False,1)
        model,history,weight_list_per_epoch = train(model,30,False)
    a,b = count_model_params_flops(model,False)
    print(a,b)
    

    validation_accuracy = max(history.history['val_accuracy'])

    best_acc_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
    log_dict['train_loss'].append(history.history['loss'][best_acc_index])
    log_dict['train_acc'].append(history.history['accuracy'][best_acc_index])
    log_dict['val_loss'].append(history.history['val_loss'][best_acc_index])
    log_dict['val_acc'].append(history.history['val_accuracy'][best_acc_index])
    a,b = count_model_params_flops(model,False)
    log_dict['total_params'].append(a)
    log_dict['total_flops'].append(b)
    log_dict['filters_in_conv1'].append(model.layers[1].get_weights()[0].shape[-1])
    log_dict['filters_in_conv2'].append(model.layers[3].get_weights()[0].shape[-1])
    print("VALIDATION ACCURACY AFTER {} ITERATIONS = {}".format(count+1,validation_accuracy))
    count+=1


Initial Validation Accuracy = 0.9926999807357788
431080 4586000
ITERATION 1 
Deleting 9/20 channels from layer: conv2d_1
Deleting 21/50 channels from layer: conv2d_2
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
 - 2s - loss: 0.0192 - accuracy: 0.9939 - val_loss: 0.0237 - val_accuracy: 0.9922
Epoch 2/10
 - 2s - loss: 0.0095 - accuracy: 0.9970 - val_loss: 0.0309 - val_accuracy: 0.9904
Epoch 3/10
 - 2s - loss: 0.0070 - accuracy: 0.9976 - val_loss: 0.0322 - val_accuracy: 0.9902
Epoch 4/10
 - 2s - loss: 0.0062 - accuracy: 0.9981 - val_loss: 0.0318 - val_accuracy: 0.9904
Epoch 5/10
 - 2s - loss: 0.0045 - accuracy: 0.9985 - val_loss: 0.0249 - val_accuracy: 0.9922
Epoch 6/10
 - 2s - loss: 0.0070 - accuracy: 0.9976 - val_loss: 0.0332 - val_accuracy: 0.9912
Epoch 7/10
 - 2s - loss: 0.0042 - accuracy: 0.9987 - val_loss: 0.0293 - val_accuracy: 0.9922
Epoch 8/10
 - 2s - loss: 0.0034 - accuracy: 0.9988 - val_loss: 0.0476 - val_accuracy: 0.9881
Epoch 9/10
 - 2s - loss: 0.0047 - accura

In [12]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 7)         182       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 17)          2992      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 multiple                  0         
_________________________________________________________________
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               1365

In [29]:
l1_norms = my_get_l1_norms_filters_per_epoch(weight_list_per_epoch)
distance_matrix_list = my_get_distance_matrix_list(l1_norms)
episodes_for_all_layers = my_get_episodes_for_all_layers(distance_matrix_list,95)
filter_pruning_indices = my_get_filter_pruning_indices(episodes_for_all_layers,l1_norms)
print(filter_pruning_indices[0][:1],filter_pruning_indices[1][:6])

[1] [1, 2, 3]


In [30]:
# all_conv_layers = my_get_all_conv_layers(model,first_time)

surgeon = Surgeon(model)
surgeon.add_job('delete_channels',model.layers[1],channels = filter_pruning_indices[0][:1])
surgeon.add_job('delete_channels',model.layers[3],channels =filter_pruning_indices[1][:1])
model = surgeon.operate()

Deleting 1/3 channels from layer: conv2d_1
Deleting 1/4 channels from layer: conv2d_2


In [31]:
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 2)         52        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 3)           153       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 multiple                  0         
_________________________________________________________________
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               2450

In [32]:
model,history,weight_list_per_epoch = train(model,60,False)

best_acc_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
log_dict['train_loss'].append(history.history['loss'][best_acc_index])
log_dict['train_acc'].append(history.history['accuracy'][best_acc_index])
log_dict['val_loss'].append(history.history['val_loss'][best_acc_index])
log_dict['val_acc'].append(history.history['val_accuracy'][best_acc_index])
a,b = count_model_params_flops(model,False)
log_dict['total_params'].append(a)
log_dict['total_flops'].append(b)
log_dict['filters_in_conv1'].append(model.layers[1].get_weights()[0].shape[-1])
log_dict['filters_in_conv2'].append(model.layers[3].get_weights()[0].shape[-1])
print("Final Validation Accuracy = ",(max(history.history['val_accuracy'])*100))

Train on 60000 samples, validate on 10000 samples
Epoch 1/60
 - 3s - loss: 0.2085 - accuracy: 0.9602 - val_loss: 0.1266 - val_accuracy: 0.9689
Epoch 2/60
 - 2s - loss: 0.0705 - accuracy: 0.9796 - val_loss: 0.0966 - val_accuracy: 0.9745
Epoch 3/60
 - 2s - loss: 0.0473 - accuracy: 0.9853 - val_loss: 0.0916 - val_accuracy: 0.9761
Epoch 4/60
 - 2s - loss: 0.0350 - accuracy: 0.9884 - val_loss: 0.0846 - val_accuracy: 0.9781
Epoch 5/60
 - 2s - loss: 0.0271 - accuracy: 0.9911 - val_loss: 0.0870 - val_accuracy: 0.9796
Epoch 6/60
 - 2s - loss: 0.0215 - accuracy: 0.9927 - val_loss: 0.0862 - val_accuracy: 0.9792
Epoch 7/60
 - 2s - loss: 0.0170 - accuracy: 0.9943 - val_loss: 0.0849 - val_accuracy: 0.9800
Epoch 8/60
 - 2s - loss: 0.0136 - accuracy: 0.9956 - val_loss: 0.0883 - val_accuracy: 0.9794
Epoch 9/60
 - 2s - loss: 0.0117 - accuracy: 0.9961 - val_loss: 0.0887 - val_accuracy: 0.9807
Epoch 10/60
 - 2s - loss: 0.0101 - accuracy: 0.9964 - val_loss: 0.0920 - val_accuracy: 0.9802
Epoch 11/60
 - 2s -

In [33]:
final_flops = count_model_params_flops(model,False)[1]
print("Total reduction in Flops in Conv layers = {}%".format(((initial_flops-final_flops)/initial_flops)*100))

Total reduction in Flops in Conv layers = 97.06061927605757%


In [34]:
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 2)         52        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 3)           153       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 multiple                  0         
_________________________________________________________________
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               2450